### Install packages

In [1]:
# !pip install -U pip
# !pip install -U setuptools wheel
# !pip install "numpy<2.0.0"  # Fix compatibility issue with AutoGluon
# !pip install -U "mxnet<2.0.0" bokeh==2.0.1
# !pip install autogluon --no-cache-dir
# !pip install kaggle

### Setup Kaggle API Key

In [2]:
# import os
# kaggle_dir = os.path.expanduser("~/.kaggle")
# !mkdir -p {kaggle_dir}
# !touch {kaggle_dir}/kaggle.json
# !chmod 600 {kaggle_dir}/kaggle.json

In [3]:
# Fill in your user name and key from creating the kaggle account and API token file
import json
import os
kaggle_username = "vbustillo"
kaggle_key = "4a62006bdac26949783ce007d3ed2c8e"

# Save API token to the kaggle.json file in the user's home directory
kaggle_path = os.path.expanduser("~/.kaggle/kaggle.json")
with open(kaggle_path, "w") as f:
    f.write(json.dumps({"username": kaggle_username, "key": kaggle_key}))

### Download and explore dataset

In [ ]:
# Download the dataset, it will be in a .zip file so you'll need to unzip it as well.
# !kaggle competitions download -c bike-sharing-demand
# If you already downloaded it you can use the -o command to overwrite the file
# !unzip -o bike-sharing-demand.zip

### Import Packages

In [4]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

/Users/victorbustillo/Documents/D01 GitHub/cd0385-project-starter/venv39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read data and transform it accordingly

In [6]:
# Extract hour, day, and month features from datetime
# Re-read the data to ensure we have the original features
train = pd.read_csv('train.csv', parse_dates=['datetime'])
test = pd.read_csv('test.csv', parse_dates=['datetime'])

# Remove casual and registered columns from training data
train = train.drop(columns=['casual', 'registered'])

# Create new time-based features
train['hour'] = train['datetime'].dt.hour
train['month'] = train['datetime'].dt.month
train['dayofweek'] = train['datetime'].dt.dayofweek

test['hour'] = test['datetime'].dt.hour
test['month'] = test['datetime'].dt.month
test['dayofweek'] = test['datetime'].dt.dayofweek

train['season'] = train['season'].astype('category')
train['weather'] = train['weather'].astype('category')
train['hour'] = train['hour'].astype('category')
train['month'] = train['month'].astype('category')
train['dayofweek'] = train['dayofweek'].astype('category')

test['season'] = test['season'].astype('category')
test['weather'] = test['weather'].astype('category')
test['hour'] = test['hour'].astype('category')
test['month'] = test['month'].astype('category')
test['dayofweek'] = test['dayofweek'].astype('category')

In [7]:
# Use log transformation approach - this is mathematically equivalent to RMSLE
# but avoids the metric calculation issues

def create_log_transformed_data(train_df):
    """Transform the target variable to log space"""
    train_log = train_df.copy()
    # Add 1 to avoid log(0), then take log
    train_log['count'] = np.log1p(train_log['count'])
    return train_log

def inverse_log_transform(predictions):
    """Transform predictions back from log space"""
    # Use expm1 which is the inverse of log1p
    return np.expm1(predictions)


In [8]:
# Apply log transformation to the target variable
train_log = create_log_transformed_data(train)

print("New features created and log transformation applied!")
print(f"Train shape: {train_log.shape}")
print(f"Test shape: {test.shape}")
print(f"Log-transformed target range: {train_log['count'].min():.4f} to {train_log['count'].max():.4f}")
train_log.head()

New features created and log transformation applied!
Train shape: (10886, 13)
Test shape: (6493, 12)
Log-transformed target range: 0.6931 to 6.8855


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,month,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,2.833213,0,1,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,3.713572,1,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,3.496508,2,1,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,2.639057,3,1,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0.693147,4,1,5


## Step 6: Hyper parameter optimization
* There are many options for hyper parameter optimization.
* Options are to change the AutoGluon higher level parameters or the individual model hyperparameters.
* The hyperparameters of the models themselves that are in AutoGluon. Those need the `hyperparameter` and `hyperparameter_tune_kwargs` arguments.

In [9]:
predictor_new_hpo = TabularPredictor(
    label="count",
    eval_metric="root_mean_squared_error",
    path='autogluon_models/bike_sharing_clean_nb_01'
).fit(
    train_data=train_log, 
    time_limit=1800, 
    presets='best_quality',
    # Simple hyperparameter configuration - just specify different models to include
    hyperparameters={
        'GBM': {},  # Use default LightGBM with AutoGluon's automatic tuning
        'XGB': {},  # Use default XGBoost with AutoGluon's automatic tuning
        'CAT': [
            {
                'iterations': 1000,
                'learning_rate': 0.05,
                'depth': 6,
                'l2_leaf_reg': 3,
                'bagging_temperature': 1,
                'random_strength': 1,
                'verbose': 0
            },
            {
                'iterations': 2000,
                'learning_rate': 0.01,
                'depth': 6,
                'l2_leaf_reg': 5,
                'bagging_temperature': 1.5,
                'random_strength': 1.5,
                'verbose': 0
            }
        ],  # Use CatBoost
        'NN_TORCH': [    # Neural Network with different configurations
            {'num_layers': 2, 'dropout_prob': 0.1, 'hidden_size': 64},
            {'num_layers': 3, 'dropout_prob': 0.2, 'hidden_size': 128},
            {'num_layers': 4, 'dropout_prob': 0.3, 'hidden_size': 256}

        ],
        'RF': [    # Random Forest with different configurations
            {'n_estimators': 100, 'max_depth': 15, 'max_features': 0.8},
            {'n_estimators': 200, 'max_depth': 20, 'max_features': 0.9},
        ],
        'XT': [    # Extra Trees with different configurations
            {'n_estimators': 100, 'max_depth': 15},
            {'n_estimators': 200, 'max_depth': 20},
        ]
    },
    verbosity=2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.19
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Thu Apr 24 20:29:18 PDT 2025; root:xnu-10063.141.1.705.2~1/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       5.20 GB / 16.00 GB (32.5%)
Disk Space Avail:   185.54 GB / 460.43 GB (40.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the da

(_ray_fit pid=30074) [1000]	valid_set's rmse: 0.258184


(_dystack pid=30062) 	-0.273	 = Validation score   (-root_mean_squared_error)
(_dystack pid=30062) 	1.83s	 = Training   runtime
(_dystack pid=30062) 	0.62s	 = Validation runtime
(_dystack pid=30062) Fitting model: RandomForest_BAG_L1 ... Training model for up to 293.02s of the 442.33s of remaining time.
(_dystack pid=30062) /Users/victorbustillo/Documents/D01 GitHub/cd0385-project-starter/venv39/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
(_dystack pid=30062)   warnings.warn(
(_dystack pid=30062) 	-0.2981	 = Validation score   (-root_mean_squared_error)
(_dystack pid=30062) 	0.58s	 = Training   runtime
(_dystack pid=30062) 	0.08s	 = Validation runtime
(_dystack pid=30062) Fitting model: RandomForest_2_BAG_L1 ... Training model for up to 292.34s of the 441.65s 

In [10]:
# Make predictions with the new feature model
predictions_log_clean_nb = predictor_new_hpo.predict(test, as_pandas=True)

# Transform predictions back from log space
predictions_clean_nb = inverse_log_transform(predictions_log_clean_nb)

print(f"Predictions range: {predictions_clean_nb.min():.2f} to {predictions_clean_nb    .max():.2f}")


Predictions range: 1.68 to 883.51


In [12]:
predictor_new_hpo.fit_summary(verbosity=1, show_plot=False)

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val              eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3  -0.259709  root_mean_squared_error       2.087677  357.117467                0.001058           0.022272            3       True         24
1       WeightedEnsemble_L2  -0.261492  root_mean_squared_error       0.995273  289.132828                0.000293           0.028136            2       True         12
2         CatBoost_2_BAG_L2  -0.262659  root_mean_squared_error       1.766087  341.358137                0.066022          26.253704            2       True         17
3           CatBoost_BAG_L2  -0.262757  root_mean_squared_error       1.735993  327.917949                0.035928          12.813516            2       True         16
4       ExtraTrees_2_BAG_L2  -0.263449  root_mean_squared_error       1.900553  316.141609   

{'model_types': {'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForest_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForest_2_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'CatBoost_2_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTrees_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTrees_2_BAG_L1': 'StackerEnsembleModel_XT',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'NeuralNetTorch_2_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'NeuralNetTorch_3_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'RandomForest_BAG_L2': 'StackerEnsembleModel_RF',
  'RandomForest_2_BAG_L2': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L2': 'StackerEnsembleModel_CatBoost',
  'CatBoost_2_BAG_L2': 'StackerEnsembleModel_CatBoost',
  'ExtraTrees_BAG_L2': 'Stac

In [13]:
# Create submission dataframe for new features model
submission_clean_nb = pd.read_csv('sampleSubmission.csv')
submission_clean_nb["count"] = predictions_clean_nb
submission_clean_nb.to_csv("submission_clean_nb.csv", index=False)

In [14]:
!kaggle competitions submit -c bike-sharing-demand -f submission_clean_nb.csv -m "clean notebook"

100%|████████████████████████████████████████| 188k/188k [00:03<00:00, 55.7kB/s]
Successfully submitted to Bike Sharing Demand

In [15]:
!kaggle competitions submissions -c bike-sharing-demand | tail -n +1 | head -n 6

fileName                        date                        description                                status                     publicScore  privateScore  
------------------------------  --------------------------  -----------------------------------------  -------------------------  -----------  ------------  
submission_clean_nb.csv         2025-07-26 20:53:13         clean notebook                             SubmissionStatus.COMPLETE  0.41055      0.41055       
submission_new_hpo.csv          2025-07-26 15:07:49         hyperparameter optimization submission     SubmissionStatus.COMPLETE  0.41055      0.41055       
submission_new_features.csv     2025-07-26 14:49:28         new features local                         SubmissionStatus.COMPLETE  1.03018      1.03018       
submission_new_hpo.csv          2025-07-26 13:57:44.233000  new features with hyperparameters          SubmissionStatus.COMPLETE  0.47026      0.47026       
